In [1]:
import os
import numpy as np
import PyPDF2
import fitz
from PyPDF2 import PdfFileMerger 
from PyPDF2 import PdfFileReader, PdfFileWriter
import re

'''Created the following folders:
"PDF Files" - main folder
"merged" - subfolder
result" - subfolder
"highlighted" - subfolder
"final" - subfolder

**********SAVE PDFs TO BE ANALYZED IN MAIN "PDF Files" FOLDER*********
'''


folder = '/home/jovyan/PDF Analyzer/'
result_file_folder = '/home/jovyan/PDF Files/result/'
merged_file_folder = '/home/jovyan/PDF Files/merged/'
highlighted_file_folder = '/home/jovyan/PDF Files/highlighted/'
highlighted_pages_file_folder = '/home/jovyan/PDF Files/highlighted page files/'
final_highlighted_file_folder = '/home/jovyan/PDF Files/final/'

os.chdir('/home/jovyan/PDF Files/')

######Enter the keyword or phrase to be searched below#########
keyword_phrase = "Mission support programs"

In [2]:
######################################   Merges all original PDFs in the above folder and will be put in merged folder
x = [a for a in os.listdir(folder) if a.endswith(".pdf")]
merger = PdfFileMerger()
for pdf in x:
    merger.append(open(pdf, 'rb'))
with open("{}{} result.pdf".format(merged_file_folder,keyword_phrase), "wb") as fout:
    merger.write(fout)

In [3]:
##################################### Get list of page numbers with the string
obj = PyPDF2.PdfFileReader("{}{} result.pdf".format(merged_file_folder,keyword_phrase))
NumPages = obj.getNumPages()

lst = []
for i in range(0, NumPages):
    PageObj = obj.getPage(i)
    Text = PageObj.extractText()
    if re.search(keyword_phrase,Text):
          lst.append(i)

In [4]:
#####################################Highlight the merged file where strings are and will be put in highlighted folder  
doc = fitz.open("{}{} result.pdf".format(merged_file_folder,keyword_phrase).format(keyword_phrase))        
for i in lst:
    page = doc[i] 
    ### SEARCH    
    text = keyword_phrase
    text_instances = page.searchFor(text)    
    ### HIGHLIGHT    
    for inst in text_instances:
        highlight = page.addHighlightAnnot(inst)     
    ### OUTPUT    
doc.save("{}{} highlighted.pdf".format(highlighted_file_folder,keyword_phrase))#, garbage=4, deflate=True, clean=True) 
inputpdf = PdfFileReader(open("{}{} highlighted.pdf".format(highlighted_file_folder,keyword_phrase), "rb"))

FitzDeprecation: 'searchFor' removed from class 'Page' after v1.19 - use 'search_for'. [__init__.py:170]
FitzDeprecation: 'addHighlightAnnot' removed from class 'Page' after v1.19 - use 'add_highlight_annot'. [__init__.py:170]


In [ ]:
##########################################Creat new PDFs with only pages with string highlights and will be put in the highlighted page files folder
inputpdf = PdfFileReader(open("{}{} highlighted.pdf".format(highlighted_file_folder,keyword_phrase), "rb"))
arr = np.array(lst)
for i in arr:
    output = PdfFileWriter()
    output.addPage(inputpdf.getPage(i))
    with open("{}{}pdf%s.pdf".format(highlighted_pages_file_folder,keyword_phrase) % i, "wb") as outputStream:
        output.write(outputStream)

In [10]:
#####################################################Compile highlighted pages into final document and will be placed in final folder    
source_dir = highlighted_pages_file_folder
merger = PdfFileMerger()
for item in os.listdir(source_dir):
    if item.endswith('.pdf'):
        #print(item)
        merger.append(source_dir + item)

with open("{}{} Highlights.pdf".format(final_highlighted_file_folder,keyword_phrase), "wb") as fout:
    merger.write(fout) 